# DATA TRANSFORMATION - 10_urgencias
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Load

In [2]:
dcc = gf.diccionario_llaves()
dcc

{'ACT': {'id': 'ID',
  'fecha': 'FE_RESULTADO',
  'fecha_no_ok': False,
  'prefi': 'act'},
 'ACT_DESAGREGADO': {'id': 'NUMERO IDENTIFICACION',
  'fecha': 'FE_RESULTADO',
  'fecha_no_ok': False,
  'prefi': 'acd'},
 'Adherencia': {'id': 'ds_identificacion',
  'fecha': 'FE_ENTREVISTA',
  'fecha_no_ok': False,
  'prefi': 'adh'},
 'Antecedentes_familiares': {'id': 'Id',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  'prefi': 'ant'},
 'Antecedentes_patologicos': {'id': 'DS_IDENTIFICACION',
  'fecha': 'FE_ACTUALIZA',
  'fecha_no_ok': False,
  'prefi': 'ant'},
 'Ayudas_diagnosticas': {'id': 'Numero_Identificacion',
  'fecha': 'Fecha_Orden',
  'fecha_no_ok': False,
  'prefi': 'ayu'},
 'Biologicos Asma': {'id': 'Identificacion',
  'fecha': 'Fecha_Dcto',
  'fecha_no_ok': False,
  'prefi': 'bio'},
 'Calidad de vida relacioada en salud': {'id': 'Identificacion',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  'prefi': 'cal'},
 'Disnea': {'id': 'id',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  

In [3]:
modulo = 'Calidad de vida relacioada en salud'

In [4]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,fecha,Dimensiones,0_100,year,month,year_month
0,1645342,2016-09-27,Entorno,75.0,2016,9,201609
1,1645342,2016-09-27,Psicologico,81.0,2016,9,201609
2,1645342,2016-09-27,Relaciones interpersonales,100.0,2016,9,201609
3,1645342,2016-09-27,Salud fisica,25.0,2016,9,201609
4,1645342,2017-12-21,Entorno,63.0,2017,12,201712


## 2. Transform

In [5]:
# pivot table to have separate variables, nans replaced with zero

ids = ['id', 'year', 'month']

base_p = base.pivot_table(index=ids, columns=['Dimensiones'], values='0_100', aggfunc=np.sum).reset_index().fillna(0)

pre = dcc[modulo]['prefi'] + '_' 
base_p.columns = [ pre + s for s in base_p.columns]
base_p.rename( columns={ pre + 'id'   :'id', 
                         pre + 'year' :'year', 
                         pre + 'month':'month',
                         pre + 'Entorno' : pre + 'ent',
                         pre + 'Psicologico' : pre + 'psi',
                         pre + 'Relaciones interpersonales' : pre + 'rel',
                         pre + 'Salud fisica' : pre + 'fis' 
                       }, inplace=True)

base_p.head()

,id,year,month,cal_ent,cal_psi,cal_rel,cal_fis
0,502989,2017,1,63.0,69.0,50.0,63.0
1,524006,2015,9,56.0,69.0,44.0,44.0
2,524006,2015,11,100.0,94.0,75.0,94.0
3,524006,2016,3,100.0,94.0,94.0,100.0
4,524006,2016,8,100.0,94.0,94.0,100.0


## 3. Merge with ids table

In [6]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_cal = ids_mensual.merge( base_p, how='left')
base_final_cal.head()

,id,year,month,cal_ent,cal_psi,cal_rel,cal_fis
0,500547,2016,1,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN


In [7]:
base_final_cal.shape

(53280, 7)